In [ ]:
# Question Answer

In [ ]:
!pip install langchain-core langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", api_key="")

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [ ]:
chain = prompt | llm

In [ ]:
def analyze_product(product):
    result = chain.invoke({"product": product})
    return result.content.strip()

In [ ]:
products = ["Computers", "Tablets", "Mobiles"]

In [ ]:
for product in products:
    print(analyze_product(product))

In [ ]:
#Text Summarization

In [ ]:
!pip install rouge-score absl-py nltk

In [ ]:
# used for to and fro conversion from models to tokens and viceversa
import tiktoken

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")

In [ ]:
# hugging face evaluate module for metrics
!pip install evaluate
import evaluate

In [ ]:
# split text to chunks

In [ ]:
def chunk_text(text, max_tokens=2000,model="gpt-4"):
  enc = tiktoken.encoding_for_model(model)
  tokens = enc.encode(text)
  for i in range(0, len(tokens), max_tokens):
    yield enc.decode(tokens[i:i+max_tokens])

In [ ]:
# "role": "user" - user's input
# "role": "system" - sets assistant behavior

In [ ]:
def summarize_chunk(chunk):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that summarizes text."},
        {"role": "user", "content": f"Summarize this for a second-grade student:{chunk}"},
    ]
  )
  return response.choices[0].message.content

In [ ]:
def summarize_long_text(text):
  chunks = list(chunk_text(text))
  summaries = [ summarize_chunk(chunk) for chunk in chunks]
  return summarize_chunk("\n\n".join(summaries))

In [ ]:
# metrics

rogue = evaluate.load("rouge")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
text = ""
summary = summarize_long_text(text)

rouge_score = rogue.compute(predictions=[summary], references=[text])
meteor_score = meteor.compute(predictions=[summary], references=[text])
bleu_score = bleu.compute(predictions=[summary], references=[text])

In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.9/475.9 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.3
    Uninstalling langchain-core-1.1.3:
      Successfully uninstalled langchain-core-1.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-co

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate

In [5]:
llm = ChatOpenAI(api_key="", model="gpt-3.5-turbo", temperature=0)

In [ ]:
# load the hr policy docs

In [8]:
loader = TextLoader("/content/temp.txt")
docs = loader.load()

In [9]:
# split text to chunks, wach chunk has 1000 chars and there is overlap of 50 chars between each chunk

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [11]:
# convert to embeddings

In [12]:
embeddings = OpenAIEmbeddings(api_key="")

In [14]:
# store embeddings in vector db

In [ ]:
vector_db = Chroma.from_documents(chunks, embeddings, persist_directory=".hr_chroma_db")

In [ ]:
retriever = vector_db.as_retriever()

In [16]:
prompt = PromptTemplate(
    input_variables=["question", "context"],
    template = """
      You are an HR assistant. Use the following pieces of context to answer the question at the end.
      If you don't know the answer, just say that you don't know, don't try to make up an answer.

      Context: {context}
      Question: {question}
      Answer:

    """
)

In [ ]:
from langchain_classic.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query = "What is the company policy on overtime"
response = chain.invoke(query)